In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2021/train.csv
/kaggle/input/tabular-playground-series-feb-2021/test.csv


In [2]:
import seaborn as sns
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
import fastai
print(fastai.__version__)
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
train.head()

2.2.5


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [4]:
train.columns
from fastai.tabular.all import *


In [5]:
dls = TabularDataLoaders.from_df(train, path='.', y_names="target",  cat_names = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6','cat7','cat8', 'cat9'],cont_names = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5','cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12','cont13'], procs=[Categorify, FillMissing, Normalize])


In [6]:
dls.show_batch()


,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,A,B,A,C,B,B,A,D,E,F,0.709091,0.522958,0.802180,0.422017,0.252626,0.182197,0.252472,0.295340,0.137169,0.198140,0.144384,0.292866,0.237087,6.712700
1,A,A,A,C,B,D,A,D,E,F,0.649600,0.470822,0.746407,0.480909,0.920568,0.475788,0.312929,0.748642,0.582598,0.702457,0.822694,0.610470,0.755164,8.083087
2,A,B,A,C,B,D,A,D,C,F,0.749835,0.262393,0.192397,0.327444,0.579872,0.850852,0.265018,0.521060,0.715887,0.733404,0.738411,0.706288,0.656730,6.432862
3,A,B,A,C,B,D,A,E,C,B,0.616997,0.366587,0.720718,0.545105,0.796489,0.563573,0.257907,0.509947,0.760359,0.465566,0.790903,0.706447,0.728658,7.161275
4,A,B,A,A,B,B,A,E,C,L,0.775779,0.423945,0.166792,0.733397,0.276213,0.446650,0.239707,0.368230,0.313514,0.389559,0.320711,0.315094,0.300947,8.349151
5,A,B,A,C,B,D,A,E,C,G,0.617409,0.086636,0.370762,0.226864,0.860845,0.513278,0.405033,0.868344,0.819243,0.331612,0.796788,0.677930,0.754947,8.258646
6,A,B,A,D,B,B,A,E,E,I,0.241134,0.275820,0.394772,0.423854,0.243284,0.302287,0.361229,0.354245,0.311556,0.309961,0.235141,0.331336,0.295241,7.724466
7,A,A,A,C,B,D,B,D,G,F,0.484634,0.155770,0.594879,0.457515,0.920527,0.622586,0.793753,0.922837,0.848555,0.856463,0.681515,0.734135,0.772373,8.280957
8,A,B,B,C,B,D,A,E,A,L,0.243383,0.141272,0.791217,0.525733,0.944788,0.532807,0.900813,0.935871,0.824043,0.725072,0.880589,0.874249,0.732290,8.690435
9,A,B,B,C,B,A,A,D,E,L,0.552003,0.319332,0.501720,0.389869,0.888887,0.587328,0.932859,0.877109,0.346698,0.285675,0.765339,0.844525,0.169789,8.846980


In [7]:
learn = tabular_learner(dls, metrics=rmse)
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,_rmse,time
0,0.785207,0.793257,0.890650,01:08
1,0.766959,0.752841,0.867665,01:08


In [8]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,A,B,A,C,B,D,A,E,E,...,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,A,B,A,C,B,D,A,E,C,...,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,A,B,A,C,B,D,A,E,C,...,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,A,A,B,A,B,D,A,E,E,...,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,A,B,A,A,B,B,A,E,E,...,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [9]:
tdl = learn.dls.test_dl(test)
preds, _  = learn.get_preds(dl=tdl)
preds.numpy().shape

(200000, 1)

In [10]:
sample = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')
sample

,id,target
0,0,0.5
1,5,0.5
2,15,0.5
3,16,0.5
4,17,0.5
...,...,...
199995,499987,0.5
199996,499990,0.5
199997,499991,0.5
199998,499994,0.5


In [11]:
sample['target'] = preds.numpy()
sample.to_csv('answer.csv',index=None)